In [1]:
import pandas as pd
import numpy as np

# Esto es para que muestre todas las columnas cuando visualizamos
pd.set_option("display.max_columns", None)

Cargamos el dataset, que está en 8 ficheros. Cargo 8 dataframes

In [2]:
df1 = pd.read_csv('.\\data\\Monday-WorkingHours.pcap_ISCX.csv')
df2 = pd.read_csv('.\\data\\Tuesday-WorkingHours.pcap_ISCX.csv')
df3 = pd.read_csv('.\\data\\Wednesday-workingHours.pcap_ISCX.csv')
df4 = pd.read_csv('.\\data\\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
df5 = pd.read_csv('.\\data\\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
df6 = pd.read_csv('.\\data\\Friday-WorkingHours-Morning.pcap_ISCX.csv')
df7 = pd.read_csv('.\\data\\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
df8 = pd.read_csv('.\\data\\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

In [3]:
# Miramos cuántas filas de cada tipo tiene cada fichero (dataframe)

print('df1\n',df1.shape, '\n', df1[' Label'].value_counts())
print('df2\n',df2.shape, '\n', df2[' Label'].value_counts())
print('df3\n',df3.shape, '\n', df3[' Label'].value_counts())
print('df4\n',df4.shape, '\n', df4[' Label'].value_counts())
print('df5\n',df5.shape, '\n', df5[' Label'].value_counts())
print('df6\n',df6.shape, '\n', df6[' Label'].value_counts())
print('df7\n',df7.shape, '\n', df7[' Label'].value_counts())
print('df8\n',df8.shape, '\n', df8[' Label'].value_counts())

df1
 (529918, 79) 
 BENIGN    529918
Name:  Label, dtype: int64
df2
 (445909, 79) 
 BENIGN         432074
FTP-Patator      7938
SSH-Patator      5897
Name:  Label, dtype: int64
df3
 (692703, 79) 
 BENIGN              440031
DoS Hulk            231073
DoS GoldenEye        10293
DoS slowloris         5796
DoS Slowhttptest      5499
Heartbleed              11
Name:  Label, dtype: int64
df4
 (170366, 79) 
 BENIGN                        168186
Web Attack � Brute Force        1507
Web Attack � XSS                 652
Web Attack � Sql Injection        21
Name:  Label, dtype: int64
df5
 (288602, 79) 
 BENIGN          288566
Infiltration        36
Name:  Label, dtype: int64
df6
 (191033, 79) 
 BENIGN    189067
Bot         1966
Name:  Label, dtype: int64
df7
 (286467, 79) 
 PortScan    158930
BENIGN      127537
Name:  Label, dtype: int64
df8
 (225745, 79) 
 DDoS      128027
BENIGN     97718
Name:  Label, dtype: int64


In [4]:
# Vemos que el lunes solo tiene tráfico normal, no lo incluinos en el conjunto final
# porque tenemos un dataset muy desbalanceado, como mucho tráfico normal comparado con los ataques.

df = pd.concat([df2,df3,df4,df5,df6,df7,df8], ignore_index=True)
df.columns = df.columns.str.replace(' ','')

In [5]:
# Vemos cuántas filas y columnas va a tener nuestro dataframe final

df.shape

(2300825, 79)

In [6]:
# Vemos cuántas filas de cada tipo de etiqueta y vemos que tráfico normal hay mucho y
# de algún tipo de ataque tenemos pocos flujos

df['Label'].value_counts()
# df['Label_category'].value_counts().plot(kind='bar'); # No la uso porque se ve mal

BENIGN                        1743179
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: Label, dtype: int64

In [7]:
# Nueva etiqueta con solo ataque sí (1) y ataque no (0)

df['EasyLabel'] = 1
df.loc[df['Label']== 'BENIGN','EasyLabel'] = 0

In [8]:
df['EasyLabel'].value_counts()

0    1743179
1     557646
Name: EasyLabel, dtype: int64

Creamos una nueva etiqueta Label_category solo numérica para análisis posteriores multiclase. No ataque (0), ataque DoS Hulk (1), ...

In [9]:
#Creamos este diccionario para Label

AttackType = {'BENIGN':0,
              'DoS Hulk':1,
              'PortScan':2,
              'DDoS':3,
              'DoS GoldenEye':4,
              'FTP-Patator':5,
              'SSH-Patator':6,
              'DoS slowloris':7,
              'DoS Slowhttptest':8,
              'Bot':9,
              'Web Attack � Brute Force':10,
              'Web Attack � XSS':11,
              'Infiltration':12,
              'Web Attack � Sql Injection':13,
              'Heartbleed':14}

In [10]:
df['Label_category'] = np.nan
for key in AttackType.keys():
    df.loc[df['Label']== key,'Label_category'] = AttackType[key]

In [11]:
df['Label'].value_counts()

BENIGN                        1743179
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: Label, dtype: int64

In [12]:
df['Label_category'].value_counts()

0.0     1743179
1.0      231073
2.0      158930
3.0      128027
4.0       10293
5.0        7938
6.0        5897
7.0        5796
8.0        5499
9.0        1966
10.0       1507
11.0        652
12.0         36
13.0         21
14.0         11
Name: Label_category, dtype: int64

In [13]:
# Comprobamos si tenemos variables con todo a 0 o valores faltantes (nan) haciendo un descriptivo
# de cada variable del dataset

df.describe()

,DestinationPort,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthMean,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMin,BwdPacketLengthMean,BwdPacketLengthStd,FlowBytes/s,FlowPackets/s,FlowIATMean,FlowIATStd,FlowIATMax,FlowIATMin,FwdIATTotal,FwdIATMean,FwdIATStd,FwdIATMax,FwdIATMin,BwdIATTotal,BwdIATMean,BwdIATStd,BwdIATMax,BwdIATMin,FwdPSHFlags,BwdPSHFlags,FwdURGFlags,BwdURGFlags,FwdHeaderLength,BwdHeaderLength,FwdPackets/s,BwdPackets/s,MinPacketLength,MaxPacketLength,PacketLengthMean,PacketLengthStd,PacketLengthVariance,FINFlagCount,SYNFlagCount,RSTFlagCount,PSHFlagCount,ACKFlagCount,URGFlagCount,CWEFlagCount,ECEFlagCount,Down/UpRatio,AveragePacketSize,AvgFwdSegmentSize,AvgBwdSegmentSize,FwdHeaderLength.1,FwdAvgBytes/Bulk,FwdAvgPackets/Bulk,FwdAvgBulkRate,BwdAvgBytes/Bulk,BwdAvgPackets/Bulk,BwdAvgBulkRate,SubflowFwdPackets,SubflowFwdBytes,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,EasyLabel,Label_category
count,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.299531e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2300825.0,2.300825e+06,2300825.0,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2300825.0,2300825.0,2300825.0,2300825.0,2300825.0,2300825.0,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06
mean,7.478905e+03,1.579823e+07,9.124129e+00,1.013505e+01,5.531908e+02,1.576287e+04,2.114469e+02,1.835353e+01,5.991961e+01,7.154907e+01,9.778951e+02,3.900557e+01,3.384654e+02,3.823140e+02,inf,inf,1.351205e+06,3.236837e+06,1.030148e+07,1.321363e+05,1.549267e+07,2.724045e+06,3.778569e+06,1.016421e+07,9.195875e+05,1.006866e+07,1.803126e+06,1.641801e+06,4.986882e+06,9.031930e+05,4.237176e-02,0.0,1.369074e-04,0.0,-2.894403e+04,-2.093103e+03,6.470259e+04,7.222130e+03,1.569328e+01,1.066273e+03,1.879099e+02,3.317741e+02,5.811043e+05,3.824324e-02,4.237176e-02,2.668608e-04,3.094581e-01,3.179964e-01,8.902633e-02,1.369074e-04,2.677301e-04,6.937720e-01,2.093426e+02,5.991961e+01,3.384654e+02,-2.894403e+04,0.0,0.0,0.0,0.0,0.0,0.0,9.124129e+00,5.531779e+02,1.013505e+01,1.576245e+04,6.508502e+03,1.743814e+03,4.958900e+00,-2.540647e+03,8.457227e+04,4.065386e+04,1.549771e+05,6.163359e+04,9.433560e+06,5.732621e+05,9.864630e+06,8.990073e+06,2.423678e-01,5.105199e-01
std,1.743643e+04,3.460566e+07,7.127588e+02,9.522747e+02,1.067420e+04,2.156969e+06,7.657296e+02,6.461895e+01,2.015989e+02,3.038008e+02,2.111330e+03,6.725428e+01,6.537513e+02,9.155096e+02,NaN,NaN,4.354930e+06,8.405745e+06,2.615439e+07,2.689066e+06,3.454541e+07,9.348407e+06,1.045887e+07,2.624079e+07,8.221362e+06,2.896989e+07,8.685394e+06,6.736929e+06,1.798607e+07,8.020122e+06,2.014359e-01,0.0,1.169995e-02,0.0,2.328742e+07,1.582975e+06,2.516112e+05,3.875489e+04,2.523412e+01,2.196991e+03,3.279417e+02,6.863408e+02,1.811366e+06,1.917830e-01,2.014359e-01,1.633370e-02,4.622704e-01,4.656982e-01,2.847818e-01,1.169995e-02,1.636027e-02,7.062611e-01,3.571097e+02,2.015989e+02,6.537513e+02,2.328742e+07,0.0,0.0,0.0,0.0,0.0,0.0,7.127588e+02,1.065863e+04,9.522747e+02,2.156929e+06,1.325259e+04,8.000072e+03,5.755252e+02,1.173879e+06,6.618916e+05,3

In [14]:
# Valores nulos e infinitos
# Quitamos las 3 últimas columnas que son las labels

for col in df.columns[:-3]:
    print(col, df[col].isnull().sum(), (~df[col].apply(np.isfinite)).sum())

DestinationPort 0 0
FlowDuration 0 0
TotalFwdPackets 0 0
TotalBackwardPackets 0 0
TotalLengthofFwdPackets 0 0
TotalLengthofBwdPackets 0 0
FwdPacketLengthMax 0 0
FwdPacketLengthMin 0 0
FwdPacketLengthMean 0 0
FwdPacketLengthStd 0 0
BwdPacketLengthMax 0 0
BwdPacketLengthMin 0 0
BwdPacketLengthMean 0 0
BwdPacketLengthStd 0 0
FlowBytes/s 1294 2430
FlowPackets/s 0 2430
FlowIATMean 0 0
FlowIATStd 0 0
FlowIATMax 0 0
FlowIATMin 0 0
FwdIATTotal 0 0
FwdIATMean 0 0
FwdIATStd 0 0
FwdIATMax 0 0
FwdIATMin 0 0
BwdIATTotal 0 0
BwdIATMean 0 0
BwdIATStd 0 0
BwdIATMax 0 0
BwdIATMin 0 0
FwdPSHFlags 0 0
BwdPSHFlags 0 0
FwdURGFlags 0 0
BwdURGFlags 0 0
FwdHeaderLength 0 0
BwdHeaderLength 0 0
FwdPackets/s 0 0
BwdPackets/s 0 0
MinPacketLength 0 0
MaxPacketLength 0 0
PacketLengthMean 0 0
PacketLengthStd 0 0
PacketLengthVariance 0 0
FINFlagCount 0 0
SYNFlagCount 0 0
RSTFlagCount 0 0
PSHFlagCount 0 0
ACKFlagCount 0 0
URGFlagCount 0 0
CWEFlagCount 0 0
ECEFlagCount 0 0
Down/UpRatio 0 0
AveragePacketSize 0 0
AvgFwdS

In [15]:
# Hay un par de variables con valores faltantes o infinitos: FlowBytes/s y FlowPackets/s que hemos detectado 
# Columns que tienen infinitos y nulo (la primera)

df[['FlowBytes/s', 'FlowPackets/s']].describe()

,FlowBytes/s,FlowPackets/s
count,2.299531e+06,2.300825e+06
mean,inf,inf
std,NaN,NaN
min,-2.610000e+08,-2.000000e+06
25%,1.182046e+02,2.680101e+00
50%,4.292120e+03,1.091561e+02
75%,1.600000e+05,2.380952e+04
max,inf,inf


In [16]:
# Valores infinitos los ponemos como valor max
df.loc[~np.isfinite(df['FlowBytes/s']), 'FlowBytes/s'] = df.loc[np.isfinite(df['FlowBytes/s']), 'FlowBytes/s'].max()
df.loc[~np.isfinite(df['FlowPackets/s']), 'FlowPackets/s'] = df.loc[np.isfinite(df['FlowPackets/s']), 'FlowPackets/s'].max()

# Eliminamos los valores nan sustituyéndolos por la mediana
df.loc[df['FlowBytes/s'].isnull(), 'FlowBytes/s'] = df.loc[~df['FlowBytes/s'].isnull(), 'FlowBytes/s'].median()

In [17]:
# En el descriptivo también hemos visto variables que solo tenía valores a 0
# 'BwdPSHFlags', 'BwdURGFlags', 'FwdAvgBytes/Bulk', 'FwdAvgPackets/Bulk', 'FwdAvgBulkRate', 'BwdAvgBytes/Bulk', 'BwdAvgPackets/Bulk', 'BwdAvgBulkRate'

for col in df.columns[:-3]:
    if (df[col].std()==0):
        print (col, df[col].std())

BwdPSHFlags 0.0
BwdURGFlags 0.0
FwdAvgBytes/Bulk 0.0
FwdAvgPackets/Bulk 0.0
FwdAvgBulkRate 0.0
BwdAvgBytes/Bulk 0.0
BwdAvgPackets/Bulk 0.0
BwdAvgBulkRate 0.0


In [18]:
df[['BwdPSHFlags', 'BwdURGFlags', 'FwdAvgBytes/Bulk', 'FwdAvgPackets/Bulk', 'FwdAvgBulkRate', 'BwdAvgBytes/Bulk', 'BwdAvgPackets/Bulk', 'BwdAvgBulkRate']].describe()

,BwdPSHFlags,BwdURGFlags,FwdAvgBytes/Bulk,FwdAvgPackets/Bulk,FwdAvgBulkRate,BwdAvgBytes/Bulk,BwdAvgPackets/Bulk,BwdAvgBulkRate
count,2300825.0,2300825.0,2300825.0,2300825.0,2300825.0,2300825.0,2300825.0,2300825.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Eliminamos las variables que no aportan información (todos los valores son 0)

In [19]:
col_to_eliminate = ['BwdPSHFlags', 'BwdURGFlags', 'FwdAvgBytes/Bulk', 'FwdAvgPackets/Bulk',
                    'FwdAvgBulkRate', 'BwdAvgBytes/Bulk', 'BwdAvgPackets/Bulk', 'BwdAvgBulkRate']

df.drop(col_to_eliminate, inplace=True, axis=1)
df.shape

(2300825, 73)

Exportamos los datos limpios. Este será nuestro fichero limpio sobre el que comenzaremos el entrenamiento de algoritmos

In [20]:
df.to_csv('.\\data\\TotalSemana_clean.csv')